# Planning Methods: Part II, Spring 2021

# Lab 4: Multicollinearity & Homoskedasticity

**About This Lab**

*DISCLAIMER: WE HAVE NOT COVERED THIS TOPIC IN CLASS. OUR NEXT TWO CLASSES WILL COVER THIS. THE EMPHASIS OF THIS LAB IS ON THE MECHANICS OF DETECTING THESE PROBLEMS.

* We will be running through this notebook together. If you have a clarifying question or other question of broad interest, feel free to interrupt or use a pause to unmute and ask it! If you have a question that may result in a one-on-one breakout room (think: detailed inquiry, conceptual question, or help debugging), please ask it in the chat!
* We recognize learning Python via Zoom comes with its challenges and that there are many modes of learning. Please go with what works best for you. That might be printing out the Jupyter notebook, duplicating it such that you can refer to the original, working directly in it. Up to you! There isn't a single right way.
* This lab requires that you download the following file and place it in the same directory as this Jupyter notebook:
    * `clean_property_data.csv`
* This data includes properties that were sold through a real estate site (like Zillow) between 2001 and 2006 in Bogota. There are apartments and houses, characteristics of the structure like area and bathrooms, and characteristics of the neighborhood like density and a proxy for neighborhood income which is called ses.

## Objectives
By the end of this lab, you will have reviewed how to:
>1. Call a correlation matrix
>2. Run multivariable linear regression

You will also learn how to:
>1. Use a and interpret the Variance Inflation Factor (VIF)
>2. Calculate predicted values using regression model 
>3. Detect heteroskedasticity using plots 
>4. Detect heteroskedasticity using the Breusch-Pagan test
>5. Use robust errors if needed

### 1. Import packages and  read the data

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

In [3]:
import statsmodels.stats.api as sms
import statsmodels.api as sm
from scipy.stats import pearsonr
from statsmodels.stats.outliers_influence import variance_inflation_factor 

from statsmodels.compat import lzip

In [ ]:
#Read in your data 
data =

#Explore your data


In [ ]:
#Create a subdataframe, create a binary variable called 'high_ses', and explore your dataframe
df = 
df['high_ses'] = 


### 2. Estimate a multivariable linear regression

We will first run a multivariable linear regression similar to the one we used in Lab 3. In the later sections, we will evaluate our results by checking for multicollinearity and heteroskedasticity.

In [ ]:
#Define a list of independent variables of interest 
ind_var = 

#Define our explanatory and dependent variables 
x = 
y = 

#Let's save the results of the regression as model.
#This is be helpful for functions we will use below 
model = 



### 3. Multicollinearity 

*THIS HAPPENS WHEN ONE OR MORE INDEPENDENT VARIABLES ARE CLOSELY RELATED WITH EACH OTHER. AS A RESULT, THE VARIABLE(S) DO NOT ADD INFORMATION TO YOUR ABILITY TO MAKE A BETTER GUESS ABOUT THE DEPENDENT VARIABLE. AS A RESULT, THE ESTIMATED COEFFICIENT HAS A LOT OF ERROR (THAT IS, THE T-STATISTIC IS LOW, THE P-VALUE IS HIGH, AND THE 95% CONFIDENCE INTERVAL INCLUDES ZERO).

*YOU RUN THIS TEST AS A WAY TO MAKE SURE THAT MULTICOLLINEARITY IS NOT RENDERING ONE OR MORE OF YOUR COEFFICIENTS AS INSIGNIFICANT.

*IF MULTICOLINEARITY IS HIGH, THEN YOU KNOW WHY YOUR COEFFICIENT IS NOT SIGNIFICANT. IF IT IS LOW, IT MUST BE SOMETHING ELSE...

#### 3.1 Correlation Matrix

We will first use a correlation matrix to examine multicollinearity. We would like to know if any of our explanatory variables are highly correlated to one another as this might affect our regression. Let's do this using `ind_var`.

In [ ]:
#Call up your correlation matrix 


#### 3.2 Variance Inflation Factor (VIF)

The VIF is a better way of detecting multicollinearity because it checks whether combinations of independent variables, in addition to each one, help explan the other independent variables. By contrast, the correlation matrix above only shows us pairwise correlations. It is possible that two variables (for example, house and # of baths, more closely predicts the area of a property than house or # of baths independently.  

How does the VIF work? By regressing each independent variable on all others. Python (and stata and R) does this automatically.  For the model above, it runs 7 regressions (there are 7 independent variables).  In each regression, a different independent variable becomes DEPENDENT, while all others remain independent. For example:

Reg 1: Dependent: high_ses; independent: All other independent variables
Reg 2: Dependent: house; indepdendent: All other independent variables
:
:
Reg 7: Dependent: pop_dens; indepdendent: All other independent variables


If the R2 in each regression is high, it means that all other independent variables explain the new dependent variable very well, so the new dependent variable does not bring much new information to the original regression model. That variable would have a high VIF.

Formally, the VIF for each variable i is:
VIF = 1/(1-R2), where R2 is the model where i is the dependent variable.

A VIF of 5 or greater can be problematic, suggesting that high colinearity may be the reason for the lack of significance in your coefficient. (Note: For a given independent variable, a VIF of 5 means that the R2 of the regression equation with that variable as dependent and all other as independent is 0.8 or 80%. That is, 80% of the variation in that variable is explained by all other independent variables. In other words, only 20% of the variation in the variable is unique to it, and hence its possible limited explanatory power. A higher VIF (e.g., 10) means a higher R2 for the regression witth that variable as dependent and all other as independent (e.g., R2 of 0.9 or 90%).

In terms of the commands below, it is important to note that VIF does not detect dataframes, therefore we extract the values of x using `df.values` into an array. You could also use `np.array` to do this. 

In [ ]:
#Let's verify the type of x in python


In [ ]:
#We will now obtain an array of our x values 
x_array = 

#Let's verify the type of x_array



In [ ]:
# We will now obtain a VIF for each of the coefficients in our model 
vif = [variance_inflation_factor(x_array, i) for i in range(x.shape[1])]
vif

Wow! What are these numbers?! What is going on? Let's add some labels to the data.

In [ ]:
#Let's label the VIF values that we obtained 


We can also obtain the VIF for a particular explanatory variable. Let's explore these steps a little 

In [ ]:
#We will calculate the shorthand VIF for 'high_ses'
#Create a sub-dataframe of your independent variables 
sub_ind = 

#Let's define our x and our y 
x = 
y = 

#Run your regression 


In [ ]:
#Let's now calculate the VIF using the formula VIF = 1/(1 - R^2)


### 4. Predictions 
Remember that one way to determine how well our model fit is comparing our guesses (the prediction) with actual values. Here we calculate make this comparison by calculating the difference between observed and predicted for each row of data. These are called residuals. The purpose in this case, however, isn't to determine model fit but to make sure that the model is not violating a critical assumption: That residuals are not uniformly distributed 

Earlier we estimated a linear regression which calculates the association between our y (price), and our explanatory variables. The coefficients we obtain can help us draw a regression line for price in this way: 

y = 11441 + (70994 * high_ses) + (-27823 * house) + (571 * area_m2) +(10065 * num_bath) +  (226 * pcn_green) + (-16 * thefts) + (-12 * pop_dens)

Keep in mind that we saved the results of the linear regression under the "model" variable above:  
> model = sm.OLS(y, x).fit()

In [ ]:
#Let's re-run the regression we obtained in part 2
x = df[ind_var].assign(Intercept = 1)
y = df['price_000']

model = sm.OLS(y, x).fit()
model.summary2()

In [ ]:
#To simply look at the coefffients we obtained we can use the following:


We can now try to make a  prediction. Today, we will use the first line of observed data from our dataframe as seen below:

In [ ]:
#Let's call our first line of x observations 


Our prediction will look like this:

y = 11441 + (70994 * 0) + (-27823 * 0) + (571 * 70) +(10065 * 2) +  (226 * 1.74)                   + (-16 * 39.92) + (-12 * 830.78)

In [ ]:
#We can ask python to find the predicted value of y using this first line
#of our observed values
#We can do this using our dataframe x:

#Or we can use our x_array:


In [ ]:
#How does this value compare to the observed value of price in our dataset
#Let's call up the first line of the dataset


In [ ]:
#Let's now calculate the residual of our first prediction


In [ ]:
#We can also find the predictions and residuals for the entirety of our dataset
#Let's see what our predictions are

#Let's see what our residuals are



### 4. Homoskedasticity

#### 4.1 Scedasticity Plots

Using the information we have gathered on our predicted values, and residuals, we can do a visual and a statistical test for homoskedasticity.

The visual test graphs the predicted values (also called fitted values) in the X-axis and the residuals in the Y-axis.

In [ ]:
def homosked_plots(prediction, residuals):
    #Define the figure
    
    #Define x, and y 
    
    #Create title for your figure
    
    #Create your plot, and set your labels
    
    #Formats axis number to include thousands separator
    ax.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x1, loc: "{:,}".format(int(x1))))
    ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda y1, loc: "{:,}".format(int(y1))))

    plt.show()

In [ ]:
homosked_plots(pred_mod, res_mod)

#### 4.2 Breusch-Pagan Test

In [ ]:
#Let's create a list of tests 
test_names = 

#Call the Breusch-Pagan test in python
results = sms.het_breuschpagan(model.resid, model.model.exog)

#Concatenate the test values to their names using lzip


#### 4.3 Robust Errors: Correcting for heteroscedasticity, if present 
The following is one approach to "correcting" for heteroskedasticity. Other approaches include: adding more variables, transforming the dependent variable (for example, with a log-transformation), or transforming independent variables.

In [ ]:
#Run your regression and introduce robust errors within your code 
#cov_type='HC0' introduces robust errors
model = 